<a href="https://colab.research.google.com/github/RaniaHamdy/Histone-modification/blob/master/CNN_Model_Variations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

from google.colab import drive
drive.mount('/content/gdrive')## data set in goole drive 

In [ ]:
pip install keras-self-attention

In [ ]:
# Basic packages  
import keras
from keras import backend as K
import tensorflow as tf
import pandas as pd 
import numpy as np
import os
from keras_preprocessing import image
from keras import preprocessing,datasets

# Packages for data preparation
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Reshape
from sklearn import preprocessing
from keras_preprocessing import sequence
# Package For Imbalanced Data Distribution
from imblearn.over_sampling import SMOTE 

#from keras import KerasClassifier
import keras.layers
from keras.engine.topology import Layer
from keras import layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Activation 
from tensorflow.keras.layers import Conv1D,Convolution1D, SeparableConv1D
from tensorflow.keras.layers import MaxPooling1D,AveragePooling1D,GlobalAveragePooling1D,GlobalMaxPooling1D
from tensorflow.keras.layers import SpatialDropout1D ,Dropout
from tensorflow.keras.layers import Conv2D,Convolution2D,ConvLSTM2D,Conv2DTranspose, DepthwiseConv2D
from tensorflow.keras.layers import AveragePooling2D,MaxPooling2D, GlobalMaxPooling2D,ZeroPadding2D
from tensorflow.keras.layers import Bidirectional,Activation,Flatten,Dense ,BatchNormalization
#from tensorflow.keras.layers import TimeDistribute
from keras.layers import LSTM ,RepeatVector
from tensorflow.compat.v1.keras.layers import CuDNNLSTM   # CuDNNLSTM not yet released for TF 2.0
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.python.keras import Input
from keras.utils import layer_utils
tf.compat.v1.keras.layers.Attention

# Packages for Data overfitting
from keras import regularizers
# Packages for Data evaluation and tunning
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,roc_curve, auc
from sklearn.model_selection import cross_val_score ,cross_val_predict ,cross_val_predict,GridSearchCV
#Model tuning 
from tensorflow.keras.initializers import glorot_uniform ,he_uniform,glorot_normal ,RandomNormal
from keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from keras.callbacks import EarlyStopping


# Packages for Data visualization
import matplotlib.pyplot as plt
#import scipy.misc
from matplotlib.pyplot import imshow
from tensorflow.keras.utils import plot_model

# problem in LStm graph(just for colap)
print("Tensorflow version " + tf.__version__)
K.clear_session()
tf.compat.v1.reset_default_graph()
tf.compat.v1.enable_eager_execution()
#print(tf.executing_eagerly())

from keras import initializers
from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler
checkpointer = ModelCheckpoint(filepath="model_seqHMDilated.h5",
                               verbose=0,
                              monitor='val_loss', save_weights_only=True, save_best_only=True)

#my_callbacks = [
    #tf.keras.callbacks.EarlyStopping(patience=2),
    #tf.keras.callbacks.ModelCheckpoint(filepath="/content/gdrive/My Drive/model_seqHMDilated.h5",monitor='val_loss',verbose=0, save_weights_only=True, save_best_only=True),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),]

seed =123
batch_size=200
epochs = 10
loss_list=[]
accuracy_list=[]
AUC_ev_list=[]
seed =123
np.random.seed(seed)
Cell_type=56

In [ ]:
def list_data():
    all_X_train_files=list(filter(lambda x: '_rc_x_train.csv' in x, os.listdir('/content/gdrive/My Drive/HM Dataset')))
    all_X_train_files.sort() 
    all_y_train_files=list(filter(lambda x: '_rc_y_train.csv' in x, os.listdir('/content/gdrive/My Drive/HM Dataset')))
    all_y_train_files.sort() 
    all_X_test_files=list(filter(lambda x: '_rc_x_test.csv' in x, os.listdir('/content/gdrive/My Drive/HM Dataset')))
    all_X_test_files.sort()
    all_y_test_files=list(filter(lambda x: '_rc_y_test.csv' in x, os.listdir('/content/gdrive/My Drive/HM Dataset')))
    all_y_test_files.sort() 
    return all_X_train_files,all_y_train_files,all_X_test_files,all_y_test_files


def load_data(fileindex):
    all_X_train_files,all_y_train_files,all_X_test_files,all_y_test_files= list_data()
    X_train_file = all_X_train_files[fileindex]
    y_train_file = all_y_train_files[fileindex]
    X_test_file = all_X_test_files[fileindex]
    y_test_file = all_y_test_files[fileindex]
    file_name = X_train_file
    x_train = np.loadtxt('/content/gdrive/My Drive/HM Dataset' + os.sep + X_train_file, delimiter = ",")
    x_test  = np.loadtxt('/content/gdrive/My Drive/HM Dataset'+ os.sep + X_test_file , delimiter = ",") 
    y_train = np.loadtxt('/content/gdrive/My Drive/HM Dataset' + os.sep + y_train_file, delimiter = ",")
    y_test  = np.loadtxt('/content/gdrive/My Drive/HM Dataset' + os.sep + y_test_file , delimiter = ",") 
    return x_train,x_test,y_train,y_test,file_name
  # using  data set for 1DCNN 
def split_data(fileindex): 
    x_train,x_test,y_train,y_test,file_name=load_data(fileindex)
    
    y_train= y_train[:,1] 
    y_test =  y_test[:,1]
    #split X_train 5 column(5 Histone)
    X_train = x_train[:,1:6] 
    X_test  = x_test[:,1:6]   

    #count gene numbers each gene 100 bin
    num_genes_train = X_train.shape[0] / 100
    num_genes_test  = X_test.shape[0] / 100
    #"np.split"This function divides the array into subarrays along a specified axis
    X_train = np.split(X_train, num_genes_train)
    X_test  = np.split(X_test, num_genes_test)


    X_train = np.array(X_train)
    X_test  = np.array(X_test)
    # convert data from list to array
    y_train = np.array(y_train)
    y_test  = np.array(y_test)
   # using  data set for 2DCNN 
   #X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],X_train.shape[2]))
   #X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],X_test.shape[2]))

    file_name=file_name
    print(file_name)  
    return y_train,y_test,X_train,X_test,file_name

def SMOTE(X_train,y_train):
  nsamples, nx, ny = X_train.shape
  X_train2d = X_train.reshape((nsamples,nx*ny))
  #print(X_train2d)
  from imblearn.over_sampling import SMOTE 
  sm = SMOTE(random_state = 2) 
  X_train_res, y_train_res = sm.fit_sample(X_train2d, y_train.ravel()) 
  nsamples1, nx1 = X_train_res.shape
  X_train=X_train_res.reshape((nsamples1,nx,ny))
  return X_train,y_train_res






In [ ]:

def CNN1D():
  sequence_input = Input(shape=(x_tr.shape[1],x_tr.shape[2]))
  x= Conv1D(50,5, padding='same', activation= 'relu')(sequence_input)#'same'
  x= MaxPooling1D(pool_size=(2))(x)
  x= Conv1D(40,3, padding='same' ,activation= 'relu')(x)
  x = Dropout(0.4 )(x)
  x= Conv1D(30,2, padding='same' ,activation= 'relu')(x)
  x = Dropout(0.4,)(x)
  x= Conv1D(20,2, padding='same' ,activation= 'relu')(x)
  x = Dropout(0.4)(x)

  x = Flatten()(x)
  x = Dense(128)(x)
  x = Dense(64)(x)

  preds = Dense(1,activation='sigmoid')(x)
  model = Model(sequence_input, preds)
  optimizer=tf.keras.optimizers.Adam(learning_rate= 0.005)
  #model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy',keras.metrics.AUC()])
  model.compile(loss= 'binary_crossentropy' ,optimizer=optimizer ,metrics=['accuracy',keras.metrics.AUC()])
  history=model.fit(x_tr, y_tr, batch_size=batch_size,epochs=epochs, validation_data=(x_va, y_va), verbose=1,callbacks = [checkpointer])
  #history=model.fit(x_tr, y_tr, batch_size=batch_size,epochs=epochs, validation_data=(x_va, y_va), verbose=1,callbacks = [my_callbacks])
  #model.summary()
  return history,model#, hist_loss_tr,hist_loss_va ,hist_acc_tr,hist_acc_va# model,#, val_acc
  #return model


def get_auc():
    history,model= CNN1D()
    predictions_NN_prob = model.predict(X_test)
    auc = roc_auc_score(y_test, predictions_NN_prob)
    print(auc)
    print('AUC: %.3f' % auc)
    list_of_numbers.append(auc)
    print('AUC=',list_of_numbers)
    return list_of_numbers ,history,model#,false_positive_rate_list,recall_list ,roc_auc_list


In [ ]:

def CNN1Datt():
  #x= Sequential()
  sequence_input = Input(shape=(x_tr.shape[1],x_tr.shape[2]))
  x= Conv1D(50,5, padding='same', activation= 'relu')(sequence_input)#'same'
  x= MaxPooling1D(pool_size=(2))(x)
  x= Conv1D(40,3, padding='same' ,activation= 'relu')(x)
  x = Dropout(0.5 )(x)
  x= Conv1D(30,2, padding='same' ,activation= 'relu')(x)
  x = Dropout(0.4)(x)
  x= Conv1D(20,2, padding='same' ,activation= 'relu')(x)
  x=  SeqSelfAttention(attention_width=2,
                       attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation=None)(x)
  x = Dropout(0.4)(x)

  x = Flatten()(x)
  x = Dense(128)(x)
  x = Dense(64)(x)

  preds = Dense(1,activation='sigmoid')(x)
  model = Model(sequence_input, preds)
  optimizer=tf.keras.optimizers.Adam(learning_rate= 0.005)
  model.compile(loss='binary_crossentropy',optimizer= optimizer ,metrics=['accuracy',keras.metrics.AUC()])
  history=model.fit(x_tr, y_tr, batch_size=batch_size,epochs= epochs, validation_data=(x_va, y_va), verbose=1,callbacks = [checkpointer])
  #model.summary()
  return history,model#, hist_loss_tr,hist_loss_va ,hist_acc_tr,hist_acc_va# model,#, val_acc


def get_auc():
    history,model= CNN1Datt()
    predictions_NN_prob = model.predict(X_test)
    auc = roc_auc_score(y_test, predictions_NN_prob)
    print(auc)
    print('AUC: %.3f' % auc)

    list_of_numbers.append(auc)
    print('AUC=',list_of_numbers)
    return list_of_numbers ,history,model#,false_positive_rate_list,recall_list ,roc_auc_list



In [ ]:
def evaluate(model):
  loss,accuracy,AUC_ev = model.evaluate(X_test, y_test)
  print('Test Error Rate: ', round((1-accuracy)*100, 2))
  #loss_list.append(loss)
  #accuracy_list.append(accuracy)
  AUC_ev_list.append(AUC_ev)
  #print('loss=',loss_list)
  #print('accuracy=',accuracy_list)
  print('AUC_ev=',AUC_ev_list)
  return accuracy_list,loss_list,AUC_ev_list

list_of_numbers=[]
for i in range(0, Cell_type):
    y_train,y_test,X_train,X_test,file_name= split_data(i)
    #X_train, y_train= SMOTE(X_train,y_train)
    #split train set into train-val
    x_tr, x_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
    list_of_numbers ,history,model = get_auc()
    avg_list_of_numbers = sum(list_of_numbers)/len(list_of_numbers)
    #accuracy_list,loss_list,AUC_ev_list= evaluate(model)
    #avg_loss_list = sum(loss_list)/len(loss_list)
    #avg_accuracy_list= sum(accuracy_list)/len(accuracy_list)
    #avg_loss_list= sum(loss_list)/len(loss_list)
    #avg_AUC_ev_list= sum(AUC_ev_list)/len(AUC_ev_list)
    avg = sum(list_of_numbers)/len(list_of_numbers)
    print("The average is ", round(avg,5))
    print("The avg_list_of_numbers is ", round(avg_list_of_numbers,5))
    i+1
    


In [ ]:
seed =123
np.random.seed(seed)
def CNN2D():
  #x= Sequential()
  sequence_input = Input(shape=(1,100,5))
  x= Conv2D(50, (5,5), padding='same')(sequence_input)
  x= MaxPooling2D(pool_size=(1,1))(x) 
  x= Conv2D(40, (3,3), padding='same', activation= 'relu')(x)
  x = Dropout(0.4)(x)
  x= Conv2D(30, (2,2), padding='same', activation= 'relu')(x)
  x = Dropout(0.4)(x)
  x= Conv2D(20, (2,2), padding='same', activation= 'relu')(x)
  x = Dropout(0.4)(x)
  x = Flatten()(x)
  x = Dense(128, activation="relu")(x)
  x = Dense(64, activation="relu")(x)

  x = Dropout(0.2)(x)

  preds = Dense(1,activation='sigmoid')(x)
  model = Model(sequence_input, preds)
  model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate= 0.005),metrics=['accuracy',keras.metrics.AUC()])
  #model.summary()
  model.fit(x_tr, y_tr, batch_size=batch_size,epochs=epochs, validation_data=(x_va, y_va), verbose=0,callbacks = [checkpointer])
  return model

predictions_NN_problist=[]
def get_auc():
    model= CNN2D()
    predictions_NN_prob = model.predict(X_test)
    auc = roc_auc_score(y_test, predictions_NN_prob)
    # evaluate model
    print(auc)
    print('AUC: %.3f' % auc)   
    predictions_NN_problist.append(predictions_NN_prob)
    list_of_numbers.append(auc)
    print(list_of_numbers)
    return list_of_numbers ,predictions_NN_problist,model

list_of_numbers=[]
for i in range(0, Cell_type):
    y_train,y_test,X_train,X_test,file_name= split_data(i)
    #X_train,y_train=SMOTE(X_train,y_train)
    #split train set into train-val
    x_tr, x_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
    list_of_numbers ,predictions_NN_problist,model = get_auc()
    avg = sum(list_of_numbers)/len(list_of_numbers)
    print("The average is ", round(avg,5))
    i+1
